In [2]:
import os, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Subset, Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
def make_transform(size=(64,64)):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
    ])

class RelabeledDataset(Dataset):
    def __init__(self, base, mapping, keep_names=None):
        self.base = base
        if keep_names is None:
            self.indices = list(range(len(base.samples)))
        else:
            keep = set(keep_names)
            self.indices = [i for i,(_,y) in enumerate(base.samples) if base.classes[y] in keep]
        self.mapping = mapping
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        x, y = self.base[self.indices[idx]]
        return x, self.mapping[y]

def build_binary_dataset(root, keep=("Forest","Residential"), size=(64,64)):
    base = datasets.ImageFolder(root, transform=make_transform(size))
    mapping = {base.class_to_idx[keep[0]]:0, base.class_to_idx[keep[1]]:1}
    ds = RelabeledDataset(base, mapping, keep_names=keep)
    return ds, base, mapping

def relabel_imagefolder(root, mapping, size=(64,64)):
    base = datasets.ImageFolder(root, transform=make_transform(size))
    ds = RelabeledDataset(base, mapping, keep_names=None)
    return ds, base

def make_loaders(dataset, batch_size=32, val_ratio=0.2, seed=42, num_workers=0):
    g = torch.Generator().manual_seed(seed)
    n = len(dataset)
    n_train = int((1 - val_ratio) * n)
    n_val = n - n_train
    train_ds, val_ds = random_split(dataset, [n_train, n_val], generator=g)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=g, num_workers=num_workers)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader

def indices_by_class(base, name):
    c = base.class_to_idx[name]
    return [i for i,(_,y) in enumerate(base.samples) if y == c]

In [5]:
def evaluate(model, loader, logits_fn, device):
    model.eval()
    tot = 0
    ok = 0
    cm = torch.zeros(2,2, dtype=torch.int64)
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            logits = logits_fn(model, x)
            pred = logits.argmax(1)
            ok += (pred == y).sum().item()
            tot += y.size(0)
            for t,p in zip(y.cpu(), pred.cpu()):
                cm[t, p] += 1
    acc = ok / tot if tot else float("nan")
    return acc, cm

def train_classifier(model, train_loader, val_loader, device, lr=1e-3, epochs=30, patience=10):
    ce = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=lr)
    best = float("inf")
    bad = 0
    hist = []
    for epoch in range(epochs):
        model.train()
        run = 0.0
        for x,y in train_loader:
            x,y = x.to(device), y.to(device)
            opt.zero_grad()
            logits = model(x)
            loss = ce(logits, y)
            loss.backward()
            opt.step()
            run += loss.item()
        run /= len(train_loader)
        model.eval()
        val = 0.0
        ok, tot = 0, 0
        with torch.no_grad():
            for x,y in val_loader:
                x,y = x.to(device), y.to(device)
                logits = model(x)
                loss = ce(logits, y)
                val += loss.item()
                ok += (logits.argmax(1)==y).sum().item()
                tot += y.size(0)
        val /= len(val_loader)
        acc = ok/tot
        hist.append((run, val, acc))
        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {run:.4f} | Val Loss: {val:.4f} | Val Acc: {acc:.4f}")
        if val < best:
            best = val
            bad = 0
            best_w = model.state_dict()
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping triggered.")
                break
    model.load_state_dict(best_w)
    return model, hist

def train_mtl(model, train_loader, val_loader, device, alpha=0.6, lr=1e-3, epochs=30, patience=10):
    ce = nn.CrossEntropyLoss()
    rec = nn.MSELoss()
    opt = optim.Adam(model.parameters(), lr=lr)
    best = float("inf")
    bad = 0
    hist = []
    for epoch in range(epochs):
        model.train()
        tr = 0.0
        for x,y in train_loader:
            x,y = x.to(device), y.to(device)
            opt.zero_grad()
            logits, recon = model(x)
            lc = ce(logits, y)
            lr_ = rec(recon, x)
            loss = alpha*lc + (1-alpha)*lr_
            loss.backward()
            opt.step()
            tr += loss.item()
        tr /= len(train_loader)
        model.eval()
        vl, ok, tot = 0.0, 0, 0
        with torch.no_grad():
            for x,y in val_loader:
                x,y = x.to(device), y.to(device)
                logits, recon = model(x)
                lc = ce(logits, y)
                lr_ = rec(recon, x)
                loss = alpha*lc + (1-alpha)*lr_
                vl += loss.item()
                ok += (logits.argmax(1)==y).sum().item()
                tot += y.size(0)
        vl /= len(val_loader)
        acc = ok/tot
        hist.append((tr, vl, acc))
        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {tr:.4f} | Val Loss: {vl:.4f} | Val Acc: {acc:.4f}")
        if vl < best:
            best = vl
            bad = 0
            best_w = model.state_dict()
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping triggered.")
                break
    model.load_state_dict(best_w)
    return model, hist

def train_soft_share(model, train_loader, val_loader, device, alpha=0.9, beta=0.09, gamma=0.01, mask_ratio=0.65, lr=1e-3, epochs=30, patience=10):
    ce = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=lr)
    best = float("inf")
    bad = 0
    for epoch in range(epochs):
        model.train()
        tr = 0.0
        for x,y in train_loader:
            x,y = x.to(device), y.to(device)
            m = (torch.rand_like(x[:, :1]) < mask_ratio).float()
            xm = x*(1-m)
            opt.zero_grad()
            logits, recon, zc, zr = model(xm)
            lc = ce(logits, y)
            l1 = torch.abs(recon - x)*m
            lr_ = l1.sum()/(m.sum()*x.size(1) + 1e-6)
            la = F.mse_loss(zc, zr)
            loss = alpha*lc + beta*lr_ + gamma*la
            loss.backward()
            opt.step()
            tr += loss.item()
        tr /= len(train_loader)
        model.eval()
        vl, ok, tot = 0.0, 0, 0
        with torch.no_grad():
            for x,y in val_loader:
                x,y = x.to(device), y.to(device)
                m = (torch.rand_like(x[:, :1]) < mask_ratio).float()
                xm = x*(1-m)
                logits, recon, zc, zr = model(xm)
                lc = ce(logits, y)
                l1 = torch.abs(recon - x)*m
                lr_ = l1.sum()/(m.sum()*x.size(1) + 1e-6)
                la = F.mse_loss(zc, zr)
                loss = alpha*lc + beta*lr_ + gamma*la
                vl += loss.item()
                ok += (logits.argmax(1)==y).sum().item()
                tot += y.size(0)
        vl /= len(val_loader)
        acc = ok/tot
        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {tr:.4f} | Val Loss: {vl:.4f} | Val Acc: {acc:.4f}")
        if vl < best:
            best = vl
            bad = 0
            best_w = model.state_dict()
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping triggered.")
                break
    model.load_state_dict(best_w)
    return model

def show_reconstructions(model, loader, recon_fn, device, n=6):
    model.eval()
    imgs, _ = next(iter(loader))
    imgs = imgs[:n].to(device)
    with torch.no_grad():
        recon = recon_fn(model, imgs)
    imgs = imgs.cpu().permute(0,2,3,1)
    recon = recon.cpu().permute(0,2,3,1)
    imgs = (imgs*0.5 + 0.5).clamp(0,1)
    recon = (recon*0.5 + 0.5).clamp(0,1)
    plt.figure(figsize=(12,4))
    for i in range(n):
        plt.subplot(2,n,i+1)
        plt.imshow(imgs[i])
        plt.axis("off")
        plt.subplot(2,n,i+1+n)
        plt.imshow(recon[i])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [6]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Sequential(nn.Flatten(), nn.Dropout(0.5), nn.Linear(64,2))
    def forward(self, x):
        x = self.encoder(x)
        return self.fc(x)

In [7]:
class MTL_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.cls_head = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Dropout(0.5), nn.Linear(128,2)
        )
        self.dec_head = nn.Sequential(
            nn.ConvTranspose2d(64,64,4,stride=2,padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64,32,4,stride=2,padding=1), nn.ReLU(),
            nn.Conv2d(32,3,3,padding=1), nn.Tanh()
        )
    def forward(self, x):
        z = self.shared(x)
        logits = self.cls_head(z)
        recon = self.dec_head(z)
        return logits, recon

In [8]:
class SoftShareMTL(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder_cls = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.BatchNorm2d(128), nn.ReLU()
        )
        self.encoder_rec = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.BatchNorm2d(128), nn.ReLU()
        )
        self.cls_head = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Dropout(0.5), nn.Linear(128,2))
        self.dec_head = nn.Sequential(
            nn.ConvTranspose2d(128,64,4,stride=2,padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64,32,4,stride=2,padding=1), nn.ReLU(),
            nn.Conv2d(32,3,3,padding=1), nn.Tanh()
        )
    def forward(self, x):
        zc = self.encoder_cls(x)
        zr = self.encoder_rec(x)
        logits = self.cls_head(zc)
        recon = self.dec_head(zr)
        zc_vec = torch.flatten(F.adaptive_avg_pool2d(zc, 1), 1)
        zr_vec = torch.flatten(F.adaptive_avg_pool2d(zr, 1), 1)
        return logits, recon, zc_vec, zr_vec

In [10]:
def main():
    set_seed(1337)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data_root = "../../data"
    ood_root = "../../data/OOD"

    ds, base, mapping = build_binary_dataset(data_root, keep=("Forest","Residential"), size=(64,64))
    train_loader, val_loader = make_loaders(ds, batch_size=32, val_ratio=0.2, seed=1337, num_workers=0)
    print(len(train_loader.dataset), len(val_loader.dataset))
    print(base.classes)
    all_labels = [y for _,y in ds]
    unique_labels = torch.unique(torch.tensor(all_labels))
    print("Labels présents dans le dataset :", unique_labels.tolist())

    model = CNNClassifier().to(device)
    model, hist = train_classifier(model, train_loader, val_loader, device, lr=1e-3, epochs=30, patience=10)

    ood_base = datasets.ImageFolder(ood_root, transform=make_transform((64,64)))
    ood_map = {
        ood_base.class_to_idx["Forest"]: 0,
        ood_base.class_to_idx["DenseResidential"]: 1,
        ood_base.class_to_idx["MediumResidential"]: 1
    }
    rel_ood = RelabeledDataset(ood_base, ood_map, keep_names=None)
    ood_loader = DataLoader(rel_ood, batch_size=64, shuffle=False)
    dense_loader = DataLoader(Subset(rel_ood, indices_by_class(ood_base,"DenseResidential")), batch_size=64, shuffle=False)
    medium_loader = DataLoader(Subset(rel_ood, indices_by_class(ood_base,"MediumResidential")), batch_size=64, shuffle=False)
    forest_loader = DataLoader(Subset(rel_ood, indices_by_class(ood_base,"Forest")), batch_size=64, shuffle=False)

    logits_baseline = lambda m,x: m(x)
    acc_all, cm_all = evaluate(model, ood_loader, logits_baseline, device)
    acc_forest, cm_forest = evaluate(model, forest_loader, logits_baseline, device)
    acc_dense, cm_dense = evaluate(model, dense_loader, logits_baseline, device)
    acc_medium, cm_medium = evaluate(model, medium_loader, logits_baseline, device)
    print("CNN | OOD acc:", round(acc_all,4))
    print("CNN | Forest:", round(acc_forest,4))
    print("CNN | Dense:", round(acc_dense,4))
    print("CNN | Medium:", round(acc_medium,4))
    print("CNN | CM OOD:\n", cm_all.numpy())

    model_mtl = MTL_CNN().to(device)
    model_mtl, hist_mtl = train_mtl(model_mtl, train_loader, val_loader, device, alpha=0.6, lr=1e-3, epochs=30, patience=10)
    logits_mtl = lambda m,x: m(x)[0]
    acc_all, cm_all = evaluate(model_mtl, ood_loader, logits_mtl, device)
    acc_forest, cm_forest = evaluate(model_mtl, forest_loader, logits_mtl, device)
    acc_dense, cm_dense = evaluate(model_mtl, dense_loader, logits_mtl, device)
    acc_medium, cm_medium = evaluate(model_mtl, medium_loader, logits_mtl, device)
    print("MTL | OOD acc:", round(acc_all,4))
    print("MTL | Forest:", round(acc_forest,4))
    print("MTL | Dense:", round(acc_dense,4))
    print("MTL | Medium:", round(acc_medium,4))
    print("MTL | CM OOD:\n", cm_all.numpy())
    recon_mtl = lambda m,x: m(x)[1]

    model_soft = SoftShareMTL().to(device)
    model_soft = train_soft_share(model_soft, train_loader, val_loader, device, alpha=0.9, beta=0.09, gamma=0.01, mask_ratio=0.65, lr=1e-3, epochs=30, patience=10)
    logits_soft = lambda m,x: m(x)[0]
    acc_all, cm_all = evaluate(model_soft, ood_loader, logits_soft, device)
    acc_forest, cm_forest = evaluate(model_soft, forest_loader, logits_soft, device)
    acc_dense, cm_dense = evaluate(model_soft, dense_loader, logits_soft, device)
    acc_medium, cm_medium = evaluate(model_soft, medium_loader, logits_soft, device)
    print("SoftShare | OOD acc:", round(acc_all,4))
    print("SoftShare | Forest:", round(acc_forest,4))
    print("SoftShare | Dense:", round(acc_dense,4))
    print("SoftShare | Medium:", round(acc_medium,4))
    print("SoftShare | CM OOD:\n", cm_all.numpy())
    recon_soft = lambda m,x: m(x)[1]

if __name__ == "__main__":
    main()

4800 1200
['Forest', 'OOD', 'Residential']
Labels présents dans le dataset : [0, 1]
Epoch [1/30] | Train Loss: 0.1201 | Val Loss: 0.0055 | Val Acc: 0.9983
Epoch [2/30] | Train Loss: 0.0073 | Val Loss: 0.0073 | Val Acc: 0.9975
Epoch [3/30] | Train Loss: 0.0082 | Val Loss: 0.0024 | Val Acc: 0.9992
Epoch [4/30] | Train Loss: 0.0030 | Val Loss: 0.0020 | Val Acc: 0.9992
Epoch [5/30] | Train Loss: 0.0111 | Val Loss: 0.0033 | Val Acc: 0.9983
Epoch [6/30] | Train Loss: 0.0055 | Val Loss: 0.0019 | Val Acc: 0.9992
Epoch [7/30] | Train Loss: 0.0029 | Val Loss: 0.0018 | Val Acc: 0.9992
Epoch [8/30] | Train Loss: 0.0015 | Val Loss: 0.0023 | Val Acc: 0.9992
Epoch [9/30] | Train Loss: 0.0048 | Val Loss: 0.0221 | Val Acc: 0.9933
Epoch [10/30] | Train Loss: 0.0059 | Val Loss: 0.0012 | Val Acc: 0.9992
Epoch [11/30] | Train Loss: 0.0050 | Val Loss: 0.0057 | Val Acc: 0.9983
Epoch [12/30] | Train Loss: 0.0054 | Val Loss: 0.0064 | Val Acc: 0.9975
Epoch [13/30] | Train Loss: 0.0030 | Val Loss: 0.0017 | Val A